In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from lazypredict.Supervised import LazyRegressor
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score,KFold,GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("cleaned_train.csv")
df.head()

,id,Place Code,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Store Area,Grocery Area,Frozen Area,...,Department,Bar For Salad,Florist,Coffee Bar,Ready Food,Video Store,Gross Weight,Net Weight,Package Weight,Min. Person Yearly Income
0,mc_ID_0,H11go_ZA,Dimes Off,Deluxe,8760000.00,4292400.00,yes,2842.23,2037.64,481.98,...,Household,1,1,1,1,1,28.20,26.60,1.60,10000.00
1,mc_ID_1,S04ne_WA,Budget Bargains,Supermarket,6360000.00,1971600.00,no,2814.95,2049.72,457.36,...,Snack Foods,0,0,0,0,0,16.57,14.97,1.60,50000.00
2,mc_ID_2,L05es_CA,Shelf Emptiers,Supermarket,10860000.00,4452600.00,yes,2192.32,1322.21,523.32,...,Periodicals,0,1,0,0,0,28.64,27.18,1.45,30000.00
3,mc_ID_4,M10da_YU,Sale Winners,Deluxe,11560000.00,4970800.00,no,2862.30,1872.19,593.93,...,Produce,1,1,1,1,1,12.62,9.71,2.91,50000.00
4,mc_ID_5,S03le_WA,Weekend Discount,Supermarket,5220000.00,1618200.00,yes,1970.17,1236.07,440.92,...,Household,0,0,1,0,0,15.41,13.95,1.45,30000.00


In [9]:
df.columns

Index(['id', 'Place Code', 'Promotion Name', 'Store Kind', 'Store Sales',
       'Store Cost', 'Is Recyclable?', 'Store Area', 'Grocery Area',
       'Frozen Area', 'Meat Area', 'Cost', 'Marriage', 'Gender', 'Children',
       'Degree', 'Work', 'Oreder Brand', 'Product', 'Department',
       'Bar For Salad', 'Florist', 'Coffee Bar', 'Ready Food', 'Video Store',
       'Gross Weight', 'Net Weight', 'Package Weight',
       'Min. Person Yearly Income'],
      dtype='object')

In [3]:
target = "Cost"
X = df.drop(columns=target)
y = df[target]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
ord_categorical_columns = ['Place Code', 'Promotion Name', 'Store Kind', 'Is Recyclable?',
                            'Children', 'Degree', 'Work', 'Oreder Brand',
                            'Product', 'Department']

hot_categorical_columns = ['Marriage', 'Gender']

all_categorical_columns = X.select_dtypes("object").columns

numeric_columns = X.select_dtypes("float").columns

In [6]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), ord_categorical_columns),
        ('ohe', OneHotEncoder(), hot_categorical_columns),
        ('num', StandardScaler(), numeric_columns)
    ]
)

In [7]:
model = make_pipeline(
    preprocessor,
    DecisionTreeRegressor(max_depth=20, random_state=42)
)
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['Place Code',
                                                   'Promotion Name',
                                                   'Store Kind',
                                                   'Is Recyclable?', 'Children',
                                                   'Degree', 'Work',
                                                   'Oreder Brand', 'Product',
                                                   'Department']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['Marriage', 'Gender']),
                                                 ('num', StandardScaler(),
                                                  Index(['Store Sales', 'Store Cost', 'Store Area', 'Grocery Area',
       'Frozen Area', 'Meat Area', 'Gross Weight', 'N

In [8]:
# Check if the model fitted well or not
predictions = model.predict(X_train)

mse = mean_squared_error(y_train, predictions)
rmse = np.sqrt(mse)
rmse

16.24279568376655

In [10]:
predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

66.13992962598071

In [11]:
rfmodel = make_pipeline(
    preprocessor,
    RandomForestRegressor(n_estimators=100, random_state=42)
)
rfmodel.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['Place Code',
                                                   'Promotion Name',
                                                   'Store Kind',
                                                   'Is Recyclable?', 'Children',
                                                   'Degree', 'Work',
                                                   'Oreder Brand', 'Product',
                                                   'Department']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['Marriage', 'Gender']),
                                                 ('num', StandardScaler(),
                                                  Index(['Store Sales', 'Store Cost', 'Store Area', 'Grocery Area',
       'Frozen Area', 'Meat Area', 'Gross Weight', 'N

In [12]:
# Check if the model fitted well or not
predictions = rfmodel.predict(X_train)

mse = mean_squared_error(y_train, predictions)
rmse = np.sqrt(mse)
rmse

19.280206350134836

In [13]:
predictions = rfmodel.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

53.03544924733378

In [14]:
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

scores = cross_val_score(rfmodel, X_train, y_train, cv=5, scoring=rmse_scorer)

positive_scores = -scores

print("Cross-validation RMSE scores:", positive_scores)

Cross-validation RMSE scores: [53.65738053 56.10354203 52.34148074 52.18038972 50.51044931]


In [15]:
preprocessor.fit(X_train)

X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

## Try Lazypredict

In [16]:
reg = LazyRegressor(verbose=1,ignore_warnings=False, custom_metric=None)
models,predictions = reg.fit(X_train_transformed, X_test_transformed, y_train, y_test)


  2%|██                                                                                 | 1/41 [00:01<01:18,  1.95s/it]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.1229879692040099, 'Adjusted R-Squared': 0.11924272209740783, 'RMSE': 147.04110042057997, 'Time taken': 1.9538815021514893}


  5%|████                                                                               | 2/41 [00:09<03:21,  5.16s/it]

{'Model': 'BaggingRegressor', 'R-Squared': 0.8722005560563311, 'Adjusted R-Squared': 0.8716547933063937, 'RMSE': 56.130699031326074, 'Time taken': 7.39117169380188}


  7%|██████                                                                             | 3/41 [00:09<01:53,  2.99s/it]

...

{'Model': 'TweedieRegressor', 'R-Squared': 0.012382814440567103, 'Adjusted R-Squared': 0.008165232153480528, 'RMSE': 156.03796866653374, 'Time taken': 0.23825287818908691}


 98%|████████████████████████████████████████████████████████████████████████████████  | 40/41 [23:29<00:22, 22.47s/it]

{'Model': 'XGBRegressor', 'R-Squared': 0.8527332424729093, 'Adjusted R-Squared': 0.8521043452877402, 'RMSE': 60.25433963077798, 'Time taken': 5.949170827865601}
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005732 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2080
[LightGBM] [Info] Number of data points in the train set: 22580, number of used features: 24
[LightGBM] [Info] Start training from score 519.389129


100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [23:31<00:00, 34.43s/it]

{'Model': 'LGBMRegressor', 'R-Squared': 0.8287743411010376, 'Adjusted R-Squared': 0.8280431283228215, 'RMSE': 64.97112573940814, 'Time taken': 1.879643440246582}


In [17]:
print(models)

                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
ExtraTreesRegressor                          0.89       0.90  50.83   
RandomForestRegressor                        0.89       0.89  53.05   
BaggingRegressor                             0.87       0.87  56.13   
XGBRegressor                                 0.85       0.85  60.25   
LGBMRegressor                                0.83       0.83  64.97   
HistGradientBoostingRegressor                0.82       0.82  65.80   
DecisionTreeRegressor                        0.81       0.81  68.95   
ExtraTreeRegressor                           0.74       0.74  79.50   
GradientBoostingRegressor                    0.47       0.47 113.79   
AdaBoostRegressor                            0.12       0.12 147.04   
MLPRegressor                                 0.08       0.09 150.00   
SVR                                          0.03       0.03 154.58   
NuSVR 

In [18]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Extra tree regressor

In [20]:
# Preprocess the data by removing missing values and scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_transformed)
X_test = scaler.transform(X_test_transformed)

In [25]:
# Apply PCA to reduce the dimensionality of the data
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [26]:

# Train an ExtraTreesRegressor model on the reduced dataset
exmodel = make_pipeline(ExtraTreesRegressor(random_state=42))
exmodel.fit(X_train_pca, y_train)


Pipeline(steps=[('extratreesregressor', ExtraTreesRegressor(random_state=42))])

In [27]:
# Check if the model fitted well or not
predictions = exmodel.predict(X_train_pca)

mse = mean_squared_error(y_train, predictions)
rmse = np.sqrt(mse)
rmse

7.680054705663122e-13

In [28]:
# Evaluate the performance of the model on the testing set
y_pred = exmodel.predict(X_test_pca)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse:.2f}')

RMSE: 138.25


# LGBMRegressor

In [35]:
# Preprocess the data by removing missing values and scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [38]:
# Apply pPCA to reduce the dimensionality of the data
ppca = PCA(n_components=10)
X_train_ppca = ppca.fit_transform(X_train)
X_test_ppca = ppca.transform(X_test)

In [41]:
from lightgbm import LGBMRegressor

In [ ]:

# Train an LGBMRegressor model on the reduced dataset using GridSearchCV for hyperparameter tuning
model = make_pipeline(LGBMRegressor())
param_grid = {
    'lgbmregressor__learning_rate': [0.01, 0.1],
    'lgbmregressor__n_estimators': range(50, 501, 50),
    'lgbmregressor__num_leaves': [31, 45, 61],
    'lgbmregressor__max_depth': [2, 4, 8, 16, 32],
}
grid_search = GridSearchCV(model, param_grid=param_grid, cv=5,n_jobs=-1,
    verbose=1)
grid_search.fit(X_train_pca, y_train)


In [ ]:
# Evaluate the performance of the model on the testing set
y_pred = grid_search.predict(X_test_pca)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse:.2f}')

# XGBRegressor

In [ ]:
# Preprocess the data by removing missing values and scaling the features using StandardScaler
scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.transform(X_test)

In [ ]:
# Apply PCA to reduce the dimensionality of the data
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_transformed)
X_test_pca = pca.transform(X_test_transformed)

In [ ]:
model = make_pipeline(preprocessor, XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=12, min_split_loss=5))
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error: {rmse:.2f}')

In [ ]:
param_grid = {
    'xgbregressor__learning_rate': [0.01, 0.1],
    'xgbregressor__n_estimators': range(50, 501, 50),
    'xgbregressor__max_depth': [2, 4, 8, 16, 32],
    'xgbregressor__min_split_loss': range(0, 7, 2)
    
}


grid_search = GridSearchCV(
    model,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_pca, y_train)

In [ ]:
y_pred = grid_search.predict(X_test_pca)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error: {rmse:.2f}')

In [ ]:
print(f'Best hyperparameters: {grid_search.best_params_}')


# BaggingRegressor

In [ ]:
# Preprocess the data by removing missing values and scaling the features using StandardScaler
scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.transform(X_test)

# Apply PCA to reduce the dimensionality of the data
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_transformed)
X_test_pca = pca.transform(X_test_transformed)

# Train a BaggingRegressor model on the reduced dataset using GridSearchCV for hyperparameter tuning
params = {
    'n_estimators': range(100, 801, 20),
}
BAG_RF = GridSearchCV(
    BaggingRegressor(random_state=42, n_jobs=-1),
    param_grid=params,
    cv=3,
    n_jobs=-1,
    verbose=1
)
BAG_RF.fit(X_train_pca, y_train)


In [ ]:
BAG_RF.best_params_
BAG = BAG_RF.best_estimator_
BAG

# RandomForestRegressor

In [ ]:
scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.transform(X_test)

# Apply PCA to reduce the dimensionality of the data
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_transformed)
X_test_pca = pca.transform(X_test_transformed)

In [ ]:

params = {
    'n_estimators': range(100, 501, 100),
    'max_depth': [2, 8, 16, 32, 64],
    'min_samples_split': [2,4,6],
    'min_samples_leaf': [1,2,3],
}
GS_RF = GridSearchCV(
    RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid=params,
    cv=3,
    n_jobs=-1,
    verbose=1
)
GS_RF.fit(X_train_pca, y_train)

# Evaluate the performance of the model on the testing set
y_pred = GS_RF.predict(X_test_pca)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse:.2f}')

In [ ]:
GS_RF.best_params_

In [ ]:
RF = GS_RF.best_estimator_
RF

# STacking

In [ ]:
from sklearn.ensemble import StackingRegressor, VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


In [ ]:
# Define the pipelines for each model
rf = make_pipeline(
    preprocessor,
    RandomForestRegressor(n_estimators=512, max_depth=16, n_jobs=-1, random_state=42)
)

et = make_pipeline(
    preprocessor,
    ExtraTreesRegressor(n_estimators=512, max_depth=16, n_jobs=-1, random_state=42)
)

bag = make_pipeline(
    preprocessor,
    BaggingRegressor(n_estimators=75, n_jobs=-1, random_state=42)
)

xgb = make_pipeline(
    preprocessor,
    XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=8, random_state=42)
)

lgbm = make_pipeline(
    preprocessor,
    LGBMRegressor(n_estimators=1000, learning_rate=0.05, max_depth=8, random_state=42)
)

In [ ]:
# Define the estimators for the ensemble models
estimators_pca = [
              ('rf', rf),
              ('et', et),
              ('bag', bag),
              ('xgb', xgb),
              ('lgbm', lgbm)
             ]

# Create the stacking model
stack_model_pca = StackingRegressor(estimators=estimators_pca, final_estimator=RandomForestRegressor(random_state=42)) # Train the stacking model with PCA
stack_model_pca.fit(X_train_transformed, y_train)


In [ ]:
# Check if the model fitted well or not with PCA
predictions_pca = stack_model_pca.predict(X_train_transformed)
mse_pca = mean_squared_error(y_train, predictions_pca)
rmse_pca = np.sqrt(mse_pca)
print(f'Training Root Mean Squared Error with PCA: {rmse_pca:.2f}')

In [ ]:

predictions_pca = stack_model_pca.predict(X_test_transformed)
mse_pca = mean_squared_error(y_test, predictions_pca)
rmse_pca = np.sqrt(mse_pca)
print(f'Testing Root Mean Squared Error with PCA: {rmse_pca:.2f}')



In [ ]:
# Create the voting model with PCA
voting_model_pca = VotingRegressor(estimators=estimators_pca)

voting_model_pca.fit(X_train_transformed, y_train)

In [ ]:
# Check if the model fitted well or not with PCA
predictions_pca = voting_model_pca.predict(X_train_transformed)
mse_pca = mean_squared_error(y_train, predictions_pca)
rmse_pca = np.sqrt(mse_pca)
print(f'Training Root Mean Squared Error with PCA: {rmse_pca:.2f}')

In [ ]:
predictions_pca = voting_model_pca.predict(X_test_transformed)
mse_pca = mean_squared_error(y_test, predictions_pca)
rmse_pca = np.sqrt(mse_pca)
print(f'Testing Root Mean Squared Error with PCA: {rmse_pca:.2f}')


## Extract submissions

In [ ]:
samples = pd.read_csv('data/sample_submission.csv')
samples.head()

In [ ]:
test = pd.read_csv("data/cleaned_test.csv")
test.head()

In [ ]:
test.isna().sum()

In [ ]:
y_sub_pred = exmodel.predict(test)

In [ ]:
samples["Cost"] = y_sub_pred

In [ ]:
samples.to_csv("data/submission.csv", index=False)